In [1]:
import pandas as pd
import numpy as np
from data_acquisition_and_processing import *
from training_models import *

In [2]:
faults_filepath = '../data/J1939Faults.csv'
diagnostics_filepath = '../data/VehicleDiagnosticOnboardData.csv'

In [3]:
train_df, test_df = clean_data(faults_filepath, diagnostics_filepath)
train_df.head()

Merging faults and diagnostics data...


c:\Users\irela\Documents\NSS_Projects\big-g-haulin-oats\notebooks\data_acquisition_and_processing.py:9: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  faults = pd.read_csv(faults_filepath)


Removing service locations and unnecessary columns...
Number of rows before removing service locations:
1187335
Number of rows after removing service locations:
1051146
Unnecessary columns removed:
['RecordID', 'ESS_Id', 'eventDescription', 'actionDescription', 'ecuSoftwareVersion', 'ecuSerialNumber', 'ecuModel', 'ecuMake', 'ecuSource', 'active', 'activeTransitionCount', 'faultValue', 'MCTNumber', 'Latitude', 'Longitude', 'LocationTimeStamp']
Data after removing service locations and unnecessary columns:
       EventTimeStamp   spn fmi EquipmentID  AcceleratorPedal  \
0 2015-02-21 10:47:13   111  17        1439               0.0   
1 2015-02-21 11:34:34   629  12        1439               NaN   
2 2015-02-21 11:35:31  1807   2        1369               NaN   
3 2015-02-21 11:35:33  1807   2        1369               NaN   
4 2015-02-21 11:39:41  4364  17        1674               NaN   

   BarometricPressure  CruiseControlActive  CruiseControlSetSpeed  \
0               14.21         

,EventTimeStamp,spn,EquipmentID,BarometricPressure,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,...,fmi_23,fmi_29,fmi_3,fmi_31,fmi_4,fmi_5,fmi_6,fmi_7,fmi_8,fmi_9
1001106,2018-07-20 09:31:33,1231,0105406655,-0.026563,-1.839029,0.370184,-0.644781,0.539284,0.318816,0.396278,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1001107,2018-07-20 09:42:30,1231,0105406655,-0.026563,-1.839029,0.370184,-0.644781,0.539284,0.318816,0.396278,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
358800,2016-01-31 07:12:25,629,105301976,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
927313,2018-01-26 18:58:08,37,105338729,0.316419,-2.466096,0.041213,-0.513425,-1.129315,0.257644,-0.956231,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
927314,2018-01-26 18:59:36,37,105338729,0.316419,-2.466096,0.041213,-0.513425,-1.129315,0.257644,-0.956231,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
prepare_data_for_model(train_df, test_df, target_filename='ffill_05_10')

Processing target: HoursUntilNextDerate
Saved HoursUntilNextDerate dataframes for training and testing in data/data_ready_for_models/
Processing target: DerateInNextTwoHours
Saved DerateInNextTwoHours dataframes for training and testing in data/data_ready_for_models/
Processing target: DerateInNextTwentyFourHours
Saved DerateInNextTwentyFourHours dataframes for training and testing in data/data_ready_for_models/


In [5]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

train_data_file_path = '../data/data_ready_for_models/DerateInNextTwentyFourHours_ffill_05_10_train.csv'
test_data_file_path = '../data/data_ready_for_models/DerateInNextTwentyFourHours_ffill_05_10_test.csv'
model = LogisticRegression()
model_name = 'initial_decision_tree_classifier'

predictions_filepath = f'../data/model_output/DerateInNextTwentyFourHours_ffill_05_10_{model_name}.csv'

In [9]:
test_df = pd.read_csv(test_data_file_path)
test_df.head()

,DerateInNextTwentyFourHours,EventTimeStamp,spn,EquipmentID,BarometricPressure,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,...,fmi_23,fmi_29,fmi_3,fmi_31,fmi_4,fmi_5,fmi_6,fmi_7,fmi_8,fmi_9
0,0,2019-01-08 19:39:29,639,105338729,-0.083727,-1.773036,0.425012,2.245062,-0.347159,0.665821,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,2019-01-08 19:39:29,639,105338729,-0.083727,-1.773036,0.425012,2.245062,-0.347159,0.665821,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0,2019-01-08 19:42:39,639,105338729,-0.083727,-1.773036,0.425012,2.245062,-0.347159,0.665821,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,2019-01-08 19:42:39,639,105338729,-0.083727,-1.773036,0.425012,2.245062,-0.347159,0.665821,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0,2019-01-16 19:41:28,37,105338729,-0.140890,-1.760107,0.260527,-0.546264,-1.077171,0.077285,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
predictions_df = start_mlflow_run(train_data_file_path, test_data_file_path, predictions_filepath, model)

2025/05/10 11:04:25 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
c:\Users\irela\Documents\NSS_Projects\big-g-haulin-oats\notebooks\training_models.py:47: DtypeWarning: Columns (3,13) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv(train_data_file_path)
2025/05/10 11:05:14 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\irela\anaconda3\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns 

Test precision: 0.0
Test recall: 0.0
Test F1 score: 0.0
Test accuracy: 0.9968261509953913
Model training and evaluation completed successfully.


In [10]:
tp, fp = count_predictions(predictions_df, test_df)
revenue_calc(tp, fp)

True Positives: 0
False Positives: 7
Total revenue: -3500


-3500